Post new ads or change ads one piece at a time

In [116]:
import pandas as pd
import pickle

In [117]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, UnexpectedAlertPresentException, WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
YA = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
opts = Options()
opts.add_argument("user-agent=YA")
opts.add_argument('--disable-extensions')
opts.add_argument("start-maximized")

In [4]:
#reload dataframe with cols: location, url, zip_code, ad_type
df = pd.read_pickle('clist_df')

In [5]:
#obtain login info and ad content
#*Note: user should define these variables directly with their own login info and ad content

emails = pickle.load(open('email_info','rb'))
passwords = pickle.load(open('pword_info','rb'))

post_title = pickle.load(open('post_title','rb'))
ad = pickle.load(open('ad_info','rb'))

phone_number = pickle.load(open('phone_info','rb'))
contact_name = pickle.load(open('contact_name','rb'))

In [112]:
post_title = 'SAT & ACT Expert | 10 yrs Experience | Affordable Rates!' 

In [124]:
##alternatively, obtain login info and ad content from db

import sqlite3
conn = sqlite3.connect('clist.db')
c=conn.cursor()

c.execute("select * from logins")
res=c.fetchall()

emails=[]
passwords=[]
for e,p in res:
    emails.append(e)
    passwords.append(p)

conn.close()

In [109]:
#function to login and post to a particular location
#*note: user will need to change the first two xpaths to correspond with the specific ad type they would like to post
def post(loc, email, password):
    link = df['url'][loc]
    zipcode = df['zip_code'][loc]
    driver = webdriver.Chrome()
    driver.get(link)
    driver.find_element_by_id('post').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[10]/label/span[2]').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[13]/label/span[2]').click()
    driver.find_element_by_id('inputEmailHandle').send_keys(email)
    driver.find_element_by_id('inputPassword').send_keys(password)
    driver.find_element_by_xpath('/html/body/article/section/div/div[1]/form/div[3]/button').click()
    try:
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
    except:           #for those locations with an extra landing page, just choose the first sublocation
        driver.find_element_by_xpath('/html/body/article/section/form/ul/li[1]/label').click()
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
#         import numpy as np
#         i=np.random.randint(1,4)
#         driver.find_element_by_xpath('/html/body/article/section/form/ul/li[{}]/label'.format(i)).click()

    driver.find_element_by_id('postal_code').send_keys(zipcode)
    
#     driver.find_element_by_id('PostingBody').send_keys(ad)                   # load ad from pickle
    driver.find_element_by_id('PostingBody').send_keys(Keys.CONTROL, 'v')      # paste ad from clipboard
    
    driver.find_element_by_xpath('//*[@id="contact_phone_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_text_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_phone"]').send_keys(phone_number)
    driver.find_element_by_xpath('//*[@id="contact_name"]').send_keys(contact_name)
    driver.find_element_by_xpath('//*[@id="wantamap"]').click()
    
    but = driver.find_element_by_css_selector('#postingForm > div > button')     
    ActionChains(driver).send_keys_to_element(but,Keys.ENTER).perform()

    driver.find_element_by_xpath('/html/body/article/section/form/button').click()

    #*option: to automatically post without first reviewing the content, uncomment the line below
#     driver.find_element_by_xpath('//*[@id="publish_top"]/button').click()

In [ ]:
def cpost(loc, email, password):
    link = df['url'][loc]
    zipcode = df['zip_code'][loc]
    driver = webdriver.Chrome()
    driver.get(link)
    driver.find_element_by_id('post').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[10]/label/span[2]').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[13]/label/span[2]').click()
    driver.find_element_by_id('inputEmailHandle').send_keys(email)
    driver.find_element_by_id('inputPassword').send_keys(password)
    try:
        driver.find_element_by_xpath('/html/body/article/section/div/div[1]/form/div[3]/button').click()
    except:
        driver.find_element_by_xpath('//*[@id="recaptcha-anchor"]/div[5]').click()
    try:
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
    except:          
        driver.find_element_by_xpath('/html/body/article/section/form/ul/li[1]/label').click()
        driver.find_element_by_id('PostingTitle').send_keys(post_title)

    driver.find_element_by_id('postal_code').send_keys(zipcode)
    
#     driver.find_element_by_id('PostingBody').send_keys(ad)                 
    driver.find_element_by_id('PostingBody').send_keys(Keys.CONTROL, 'v')    
    
    driver.find_element_by_xpath('//*[@id="contact_phone_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_text_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_phone"]').send_keys(phone_number)
    driver.find_element_by_xpath('//*[@id="contact_name"]').send_keys(contact_name)
    driver.find_element_by_xpath('//*[@id="wantamap"]').click()
    
    but = driver.find_element_by_css_selector('#postingForm > div > button')     
    ActionChains(driver).send_keys_to_element(but,Keys.ENTER).perform()

    driver.find_element_by_xpath('/html/body/article/section/form/button').click()
#     driver.find_element_by_xpath('//*[@id="publish_top"]/button').click()

### Next steps: 
1. forget about separating into hs and college
2. separate out those w extra landing pages
2. fix zip codes

In [75]:
locs=list(df.index)

In [113]:
from itertools import cycle
it= cycle(locs)

In [128]:
for e,p in res:
    for i in range(2):
        post(next(it),e,p)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/article/section/form/ul/li[1]/label"}
  (Session info: chrome=62.0.3202.94)
  (Driver info: chromedriver=2.30.477700 (0057494ad8732195794a7b32078424f92a5fce41),platform=Windows NT 6.3.9600 x86_64)


In [130]:
post(next(it),emails[0],passwords[0])

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/article/section/form/ul/li[1]/label"}
  (Session info: chrome=62.0.3202.94)
  (Driver info: chromedriver=2.30.477700 (0057494ad8732195794a7b32078424f92a5fce41),platform=Windows NT 6.3.9600 x86_64)
